## Вычисление Recall, IoU по нашему датасету

In [2]:
import os
import cv2
import numpy as np
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from collections import defaultdict

In [3]:
# Настройка модели
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

In [4]:
# Список классов в датасете COCO
coco_classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
print(coco_classes)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [5]:
# Список классов
classes = ['bicycle', 'bus', 'car', 'motorbike', 'person']

# Перевод id меток датасета в id COCO для последующего сравнения по id 
class_indices = []
for classs in classes:
    try:
        class_indices.append(coco_classes.index(classs))
    except ValueError:
        print(f"Warning: Class '{classs}' not found in COCO classes.")
        class_indices.append(None)

In [6]:
# Вычисление IoU по 2-м bb
def compute_iou(box1, box2):
    _, x1, y1, x2, y2 = box1
    _, x1_b, y1_b, x2_b, y2_b = box2

    # Вычисление пересечения
    x1_max = max(x1, x1_b)
    y1_max = max(y1, y1_b)
    x2_min = min(x2, x2_b)
    y2_min = min(y2, y2_b)

    intersection = max(0, x2_min - x1_max) * max(0, y2_min - y1_max)

    # Вычисление объединения
    area_box1 = (x2 - x1) * (y2 - y1)
    area_box2 = (x2_b - x1_b) * (y2_b - y1_b)
    union = area_box1 + area_box2 - intersection

    # Вычисление IoU
    iou = intersection / union if union != 0 else 0

    return iou

# Функция для чтения меток из txt файла
def read_labels(txt_file):
    with open(txt_file, "r") as f:
        lines = f.readlines()
    boxes = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        # Преобразование координат из YOLO формата в формат xmin, ymin, xmax, ymax
        xmin = (x_center - width / 2) * img.shape[1]
        xmax = (x_center + width / 2) * img.shape[1]
        ymin = (y_center - height / 2) * img.shape[0]
        ymax = (y_center + height / 2) * img.shape[0]
        boxes.append([class_id, xmin, ymin, xmax, ymax])
    return boxes

# Получить ответ модели в таком же формате, что и в датасете
def get_predicts(outputs):
    # Извлечение предсказанных ограничивающих рамок и классов из outputs
    pred_boxes = outputs["instances"].pred_boxes.tensor.tolist()
    pred_classes = outputs["instances"].pred_classes.tolist()
    
    # Компонуем все вместе, как в метках нашего даатсета
    predicts = []
    for pred_class, pred_box in zip(pred_classes, pred_boxes):
        predicts.append([pred_class] + pred_box)
    
    return predicts


In [7]:
# Получение списка изображений
image_dir = "../test/images"
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

In [8]:
# Инициализация словарей для подсчета TPm FP и FN для каждого класса
TP = defaultdict(int)
FP = defaultdict(int)
FN = defaultdict(int)

# Список для хранения Precision и Recall для каждого класса
precisions = defaultdict(list)
recalls = defaultdict(list)

# Список для хранения IoU для каждого bb
ious = []

# Прогон каждого изображения через модель
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    img = cv2.imread(image_path)
    outputs = predictor(img)
    
    predicts = get_predicts(outputs)
    
    # Чтение истинных меток из соответствующего txt файла
    txt_file = os.path.join("../test/labels", image_file.replace(".jpg", ".txt"))
    true_boxes = read_labels(txt_file)

    # Вычисление метрик
    for true_box in true_boxes:
        max_iou = 0
        max_class_id2 = -1
        for predict in predicts:
            iou = compute_iou(true_box, predict)
            class_id1, class_id2 = class_indices[int(true_box[0])], predict[0]
            # Ищем такую пару bb, чтобы IoU было максимальным
            if iou > max_iou:
                max_iou = iou
                max_class_id2 = class_id2
        if class_id1 == max_class_id2:  # Если классы совпадают, считаем это истинно положительным предсказанием
            TP[class_id1] += 1
        else:  # В противном случае, считаем это ложно отрицательным предсказанием и ложно положительным предсказанием
            FN[class_id1] += 1
            FP[max_class_id2] += 1

        ious.append(max_iou)
        
# Вычисление precision и recall для каждого класса
precision = {classes[i]: TP[i] / (TP[i] + FP[i]) for i in range(len(classes)) if (TP[i] + FP[i]) != 0}
recall = {classes[i]: TP[i] / (TP[i] + FN[i]) for i in range(len(classes)) if (TP[i] + FN[i]) != 0}

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
# Вычисление precision и recall для каждого класса
precision = {classes[i]: TP[i] / (TP[i] + FP[i]) for i in range(len(classes)) if (TP[i] + FP[i]) != 0}
recall = {classes[i]: TP[i] / (TP[i] + FN[i]) for i in range(len(classes)) if (TP[i] + FN[i]) != 0}

In [10]:
print("Precision:", precision)
print("Recall:", recall)
print("Mean IoU:", np.mean(ious))

Precision: {'bicycle': 0.44598337950138506, 'bus': 0.5714285714285714, 'car': 0.9477993858751279, 'motorbike': 0.0}
Recall: {'bicycle': 0.3561946902654867, 'bus': 0.36363636363636365, 'car': 0.48304642670839854}
Mean IoU: 0.3555233791658838
